<a href="https://colab.research.google.com/github/BiancadeFrancisco/BigData_Fraude_transacaoPIX/blob/main/PYSPARK_CASE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


- Arquivo .json

  ```json
  {
      "id_transacao": inteiro,
      "valor": texto,
      "remetente": {
          "nome": texto,
          "banco": texto,
          "tipo": texto
      },
      "destinatario": {
          "nome": texto,
          "banco":texto,
          "tipo": texto
      },          
      "categoria":texto,
      "chave_pix":texto,
      "transaction_date":texto,
      "fraude":inteiro,
  }
    ```


# Preparação do Ambiente de Desenvolvimento

In [ ]:
# Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=df92ee84a1ce6c795da0b57a89225c6ea429fd5743c43f6a6c84ebc55c721b4e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://b404-35-233-183-41.ngrok-free.app


# Preparação dos Dados


In [ ]:
# Iniciar a sessão spark:

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .appName("CaseFinal")
      .getOrCreate()
)

In [ ]:
from pyspark.sql.types import *

caminho_json = '/content/case_final.json'

#schema = ESQUEMA/PROGRAMA/ESBOÇO

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])


schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])


# formato de data do nosso arquivo original: 2022-10-20 10:57:36

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss",
)

In [ ]:
# VER TIPOS DE DADOS:

df.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)



In [ ]:
df.show()

+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588.08|{Jonathan Gonsalv...|{Calebe Melo, Cai...|aleatoria|       outros|2021-07-16 05:00:55|     0|
|        1001|           80682.5|{Jonathan Gonsalv...|{Davi Lucas Perei...|  celular|transferencia|2022-04-20 12:34:01|     1|
|        1002|             549.9|{Jonathan Gonsalv...|{Sabrina Castro, ...|      cpf|        lazer|2022-07-10 16:51:34|     0|
|        1003|             90.83|{Jonathan Gonsalv...|{Francisco da Con...|aleatoria|   transporte|2022-10-20 10:57:36|     0|
|        1004|13272.619999999999|{Jonathan Gonsalv...|{Isabelly Ferreir...|    email|transferencia|2021-04-06 2

In [ ]:
# PRECISO FAZER UM FLATEN NA COLUNA REMETENTE E DESTINATÁIO PARA ABRIR OS DADOS QUE CONSTAM NELAS, APÓS POSSO EXCLUIR AS ORIGINAIS:

from pyspark.sql.functions import col

df_flatten = df.withColumns({
    'remetente_nome':  col('remetente').getField('nome'),
    'remetente_banco':  col('remetente').getField('banco'),
    'remetente_tipo':  col('remetente').getField('tipo'),
    'destinatario_nome':  col('destinatario').getField('nome'),
    'destinatario_banco':  col('destinatario').getField('banco'),
    'destinatario_tipo':  col('destinatario').getField('tipo'),
}).drop('remetente', 'destinatario')



In [ ]:
# VERIFICAR COMO FICARAM OS DADOS APÓS AJUSTE DAS COLUNAS REMETENTE E DESTINATÁRIO:

df_flatten.printSchema()

df_flatten.show()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- remetente_nome: string (nullable = true)
 |-- remetente_banco: string (nullable = true)
 |-- remetente_tipo: string (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)

+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|
+------------+------------------+---------+-------------+-------------

In [ ]:
# DESCREVER ESTATISTICAMENTE OS DADOS

df_flatten.describe().show()

# EXEMPLO DE LEITURA APÓS DESCRIÇÃO ESTATISTICA DOS DADOS:
  # coluna valor: há transações com no mín 0,00 e máx 89996,33
  # coluna fraude: em torno de 15% é considerado fraude
  # coluna transaction_date: os dados estão entre 14/01/2021 e 15/01/2023


+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|summary|     id_transacao|             valor|chave_pix|  categoria|   transaction_date|            fraude|    remetente_nome|remetente_banco|remetente_tipo|destinatario_nome|destinatario_banco|destinatario_tipo|
+-------+-----------------+------------------+---------+-----------+-------------------+------------------+------------------+---------------+--------------+-----------------+------------------+-----------------+
|  count|           100000|            100000|   100000|     100000|             100000|            100000|            100000|         100000|        100000|           100000|            100000|           100000|
|   mean|          50999.5|10303.358732200059|     null|       null|               null|           0.15367|              null|           null|      

# Modelagem


In [ ]:
# - Para qual banco esse cliente mais transfere?

df_flatten.groupBy('destinatario_banco').count().orderBy('count').show()

# banco com maior tranferência: XP

+------------------+-----+
|destinatario_banco|count|
+------------------+-----+
|          Bradesco|14187|
|                C6|14204|
|             Caixa|14240|
|              Itau|14281|
|            Nubank|14297|
|               BTG|14390|
|                XP|14401|
+------------------+-----+



In [ ]:
# sei que o banco que mais faz transferencia é o XP, mas qual banco é mais utilizado por mÊs?
# primeiro, preciso ajustar a data apenas para o formato de mês e ano

from pyspark.sql.functions import date_format

df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy-MM').alias('ano_mes'),
    'destinatario_banco'
).count().orderBy(col('ano_mes').desc()).show()

+-------+------------------+-----+
|ano_mes|destinatario_banco|count|
+-------+------------------+-----+
|2023-01|              Itau|  267|
|2023-01|             Caixa|  277|
|2023-01|                XP|  277|
|2023-01|          Bradesco|  280|
|2023-01|            Nubank|  290|
|2023-01|                C6|  290|
|2023-01|               BTG|  278|
|2022-12|                XP|  615|
|2022-12|               BTG|  603|
|2022-12|                C6|  576|
|2022-12|          Bradesco|  575|
|2022-12|            Nubank|  602|
|2022-12|              Itau|  633|
|2022-12|             Caixa|  616|
|2022-11|          Bradesco|  579|
|2022-11|               BTG|  580|
|2022-11|              Itau|  614|
|2022-11|            Nubank|  620|
|2022-11|             Caixa|  543|
|2022-11|                C6|  561|
+-------+------------------+-----+
only showing top 20 rows



In [ ]:
# Qual o valor médio de transações por banco?

df_flatten.groupBy(
    'destinatario_banco'
).avg('valor').orderBy('avg(valor)').show()

# consigo visualizar com esse dado, que o BTG, por exemplo é o segundo banco com mais transações, porém é o banco com transações em valores menores

+------------------+------------------+
|destinatario_banco|        avg(valor)|
+------------------+------------------+
|               BTG|10122.299803335622|
|              Itau|10230.876305580874|
|             Caixa|10254.864015449395|
|                C6|10309.499774711307|
|            Nubank|10316.475401133126|
|                XP|10328.071572113045|
|          Bradesco| 10564.19458870794|
+------------------+------------------+



In [ ]:
# Quais categorias é gasto em cada banco?

df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy-MM').alias('ano_mes'),
    'destinatario_banco',
    'categoria'
).count().orderBy('ano_mes').show()

# podemos ver que a categoria transferência possui os maiores valores gastos

+-------+------------------+-------------+-----+
|ano_mes|destinatario_banco|    categoria|count|
+-------+------------------+-------------+-----+
|2021-01|             Caixa|     educacao|   24|
|2021-01|              Itau|        saude|   40|
|2021-01|            Nubank|        saude|   28|
|2021-01|                XP|    presentes|   38|
|2021-01|               BTG|   transporte|   29|
|2021-01|               BTG|  alimentacao|   31|
|2021-01|              Itau|transferencia|   75|
|2021-01|                XP|       outros|   34|
|2021-01|                XP|   transporte|   26|
|2021-01|          Bradesco|transferencia|   96|
|2021-01|              Itau|    vestuario|   28|
|2021-01|               BTG|        lazer|   34|
|2021-01|          Bradesco|   transporte|   40|
|2021-01|                XP|transferencia|   89|
|2021-01|                C6|  alimentacao|   38|
|2021-01|             Caixa|        lazer|   37|
|2021-01|          Bradesco|       outros|   25|
|2021-01|           

In [ ]:
# soma de quanto ele gasta em cada categoria por mês

df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy').alias('ano'),
    'categoria'
).sum('valor').select('ano', 'categoria', col('sum(valor)').cast(DecimalType(38, 3)).alias('valor')).orderBy('valor').show(30)

# confirma que os valores de transferencia são os mais altos

+----+-------------+-------------+
| ano|    categoria|        valor|
+----+-------------+-------------+
|2023|    presentes|   362584.450|
|2023|  alimentacao|   392078.310|
|2023|        saude|   400683.640|
|2023|   transporte|   427790.540|
|2023|     educacao|   432305.660|
|2023|    vestuario|   459528.700|
|2023|        lazer|   469671.410|
|2023|       outros|   501308.340|
|2021|   transporte|  9497893.470|
|2021|        lazer|  9622503.140|
|2021|       outros|  9737076.380|
|2022|        saude| 10048245.070|
|2021|     educacao| 10106095.070|
|2021|  alimentacao| 10237928.250|
|2022|        lazer| 10295747.630|
|2022|    presentes| 10311585.740|
|2022|     educacao| 10367124.440|
|2021|        saude| 10384662.820|
|2021|    vestuario| 10412651.930|
|2021|    presentes| 10496042.020|
|2022|  alimentacao| 10545783.230|
|2022|   transporte| 10553408.290|
|2022|       outros| 10566645.100|
|2022|    vestuario| 10696006.480|
|2023|transferencia| 16148682.620|
|2021|transferencia|

In [ ]:
# média de transferencia por período

df_flatten.groupBy(
    date_format(col('transaction_date'), 'yyyy').alias('ano')
).avg('id_transacao').select('ano',col('avg(id_transacao)').alias('avg')).orderBy('ano').show(30)

+----+------------------+
| ano|               avg|
+----+------------------+
|2021| 51204.03597092333|
|2022|50823.494263441935|
|2023| 50481.67993874426|
+----+------------------+



In [ ]:
# quantas transações com fraude existe:

df_flatten.groupBy('fraude').count().show()


+------+-----+
|fraude|count|
+------+-----+
|     1|15367|
|     0|84633|
+------+-----+



In [ ]:
# qual categoria possui mais fraude:

df_flatten.groupBy('categoria', 'fraude').count().show()

# a unica categoria que tem fraude é a de transferência, que tem 15367 fraudes e 9377 não fraudes

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|       outros|     0| 9377|
|     educacao|     0| 9460|
|transferencia|     0| 9377|
|transferencia|     1|15367|
|    presentes|     0| 9254|
|        saude|     0| 9476|
|        lazer|     0| 9464|
|   transporte|     0| 9174|
|    vestuario|     0| 9503|
|  alimentacao|     0| 9548|
+-------------+------+-----+



In [ ]:
# filtrar categoria transferencia e verificar/confirmar o número de fraudes:

df_flatten.filter(
    col('categoria') == 'transferencia'
).groupBy('categoria', 'fraude').count().show()

+-------------+------+-----+
|    categoria|fraude|count|
+-------------+------+-----+
|transferencia|     0| 9377|
|transferencia|     1|15367|
+-------------+------+-----+



In [ ]:
# Quero saber os valores de fraude. Primeiro cria-se uma coluna range (extensão/faixa de valores) sempre de 10 mil em 10 mil reais, para então realizar a contagem de fraudes dentro dessas faixas de valores.

from pyspark.sql.functions import floor

df_flatten.filter(col('fraude') == 1).withColumn(
    "range",
    floor(col("valor")/1000)*1000
).groupBy('range').count().orderBy(col('range').desc()).show()

# leitura: podemos ver que na faixa de 89 mil, tenho 222 transações com fraude

+-----+-----+
|range|count|
+-----+-----+
|89000|  222|
|88000|  208|
|87000|  230|
|86000|  203|
|85000|  205|
|84000|  245|
|83000|  206|
|82000|  206|
|81000|  214|
|80000|  213|
|79000|  205|
|78000|  230|
|77000|  237|
|76000|  232|
|75000|  190|
|74000|  207|
|73000|  237|
|72000|  234|
|71000|  234|
|70000|  222|
+-----+-----+
only showing top 20 rows



In [ ]:
# quero saber qual o valor mínimo e máximo considerado fraude

from pyspark.sql.functions import max, min

df_flatten.filter(
    col('fraude') == 1
).withColumn(
    "range", floor(col("valor")/1000)*1000
).select(max('range'), min('range')).show()

# leitura: tive transferência considerada fraude com valores acima de 19000,00 *SERÁ QUE PRECISO BLOQUEAR TRANSFERENCIAS ACIMA DESSE VALOR, PARA EVITAR FRAUDES?)

+----------+----------+
|max(range)|min(range)|
+----------+----------+
|     89000|     19000|
+----------+----------+



## Modelo de Predição de Fraudes

In [ ]:
from pyspark.sql.functions import col, udf

from pyspark.ml import Pipeline  # VAI CRIAR UM PIPELINE (linha de processo) PARA EU EXECUTAR AS AÇÕES DO ALGORITMO
from pyspark.ml.feature import StringIndexer  # VAI TRANSFORMAR O QUE É DADO CATEGÓRICO EM NUMÉRICO (para regressão logistica utilizamos apenas números)
from pyspark.ml.feature import VectorAssembler # VAI UNIR TODAS AS COLUNAS EM APENAS UMA
from pyspark.ml.classification import LogisticRegression # NOSSO ALGORITMO DE MACHINE LEARNING PARA CLASSIFICAÇÃO

In [ ]:
# EXCLUIR ALGUMAS COLUNAS QUE NÃO SERÃO RELEVANTES EM NOSSAS ANÁLISES / CRIAR UM NOVO DATASET APENAS COM AS COLUNAS NECESSÁRIAS

df = df_flatten.drop('remetente', 'id') # nesse caso específico os dados são apenas de um remetente, então podemos excluir essa coluna

In [ ]:
df.show()

+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|
+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|            BTG|            PF|         Calebe Melo|             Caixa|               PF|
|        1001|           80682.5|  celular|transferencia|2022-04-20 12:34:01|     1|Jonathan Gonsalves|            BTG|            PF|  Davi Lucas Pereira|             Caixa|               PJ|
|        1002|             549.9|  

In [ ]:
indexer = StringIndexer(
    inputCols=[
        "destinatario_nome",
        "destinatario_banco",
        "destinatario_tipo",
        "categoria",
        "chave_pix"
    ],
    outputCols=[
        "destinatario_nome_index",
        "destinatario_banco_index",
        "destinatario_tipo_index",
        "categoria_index",
        "chave_pix_index"
    ])

In [ ]:
df_index = indexer.fit(df).transform(df)
df_index.show()

+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|    remetente_nome|remetente_banco|remetente_tipo|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------+------------------+---------+-------------+-------------------+------+------------------+---------------+--------------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|Jonathan Gonsalves|   

In [ ]:
# filtrar apenas as colunas que quero usar e criar com dataframe com elas (deve ser apenas com colunas numéricas):

cols_para_filtrar = [
  "valor",
  "transaction_date",
  "destinatario_nome_index",
  "destinatario_banco_index",
  "destinatario_tipo_index",
  "chave_pix_index",
  "categoria_index",
  "fraude"
]

In [ ]:
# SEPARAR DATAFRAME CRIADO EM FRAUDE E NÃO FRAUDE:

is_fraud = df_index.select(cols_para_filtrar).filter("fraude == 1")
no_fraud = df_index.select(cols_para_filtrar).filter("fraude == 0")


no_fraud = no_fraud.sample(False, 0.01, seed = 123)  # PEGAR AMOSTRA DE 1%

In [ ]:
# REUNIR ESSA AMOSTRA PARA O NOSSO DADO NÃO FICAR COM VIÉS, COM OS DADOS QUE SÃO FRAUDE, ASSIM TEMOS 16 MIL REGISTROS PARA NOSSO TESTE

df_concat = no_fraud.union(is_fraud)
df = df_concat.sort("transaction_date")
df.count()

16202

In [ ]:
# SEPARAR DATAFRAME DE TESTE E TREINO:

train, test = df.randomSplit([0.7, 0.3], seed = 123) #70% DE TREINO E 30% DE TESTE
print("train =", train.count(), " test =", test.count())

train = 11278  test = 4924


In [ ]:
# CRIAMOS A FUNÇÃO UDF, QUE IRÁ CRIAR UMA NOVA COLUNA CHAMADA IS_FRAUDE E CASO NOSSA COLUNA DE FRAUDE SEJA > 0, ELE VAI COLOCAR ESSA COLUNA COMO 1.0, CASO CONTRÁRIO VAI COLOCAR COMO 0.0

is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [ ]:
# CRIAR NOSSO MODELO DE MACHINE LEARNING

# Create the feature vectors.
# VectorAssembler is a transformer that combines a given list of columns into a single vector column.
assembler = VectorAssembler(
  inputCols = [x for x in train.columns if x not in ["transaction_date", "fraude", "is_fraud"]],
  outputCol = "features")

# Use Logistic Regression.
# is a machine learning algorithm that is used for classification tasks
lr = LogisticRegression().setParams(
    maxIter = 100000,
    labelCol = "is_fraud",
    predictionCol = "prediction")


# This will train a logistic regression model on the input data and return a
# LogisticRegressionModel object which can be used to make predictions on new data.
model = Pipeline(stages = [assembler, lr]).fit(train)

In [ ]:
# VERIFICAR A PREDIÇÃO DO MODELO:

predicted = model.transform(test)

predicted = predicted.withColumn("is_fraud", is_fraud(predicted.fraude))
predicted.crosstab("is_fraud", "prediction").show()

# NOSSO MODELO CONSEGUIU PREVER QUASE 100% DOS CASOS QUE SÃO FRAUDE, APENAS COM ERRO EM 2 CASOS, QUE NÃO ERAM FRAUDES, MAS A PREDIÇÃO DISSE QUE ERA FRAUDE

+-------------------+---+----+
|is_fraud_prediction|0.0| 1.0|
+-------------------+---+----+
|                1.0|  0|4660|
|                0.0|262|   2|
+-------------------+---+----+

